In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

REQUEST_URL = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=050&bs=040&ta=23&sc=23211&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1"
MAX_ITEMS = 200
results = []
page_number = 1
item_count = 0

while item_count < MAX_ITEMS:
    res = f"{REQUEST_URL}&page={page_number}"
    response = requests.get(res)
    if response.status_code != 200:
        break

    soup = BeautifulSoup(response.text, "html.parser")
    cassette_items = soup.find_all(class_="cassetteitem")

    for item in cassette_items:
        # 各cassetteitemのタイトルなど共通情報を抽出
        title = item.find(class_="cassetteitem_content-title").text.strip()
        address = item.find(class_="cassetteitem_detail-col1").text.strip()
        
        # 各cassetteitem内の複数のtbodyを取得
        tbodies = item.find_all('tbody')
        for tbody in tbodies:
            if item_count >= MAX_ITEMS:
                break
            # 各tbodyから間取り、料金、階数などの情報を抽出
            madori = tbody.find(class_="cassetteitem_madori").text.strip()
            fee = tbody.find(class_="cassetteitem_other-emphasis ui-text--bold").text.strip()
            floor = tbody.find_all('td')[2].text.strip() if tbody.find_all('td') else "不明"
            
            # 各物件の情報を辞書として追加
            bukken_info = {
                "title": title,
                "address": address,
                "madori": madori,
                "fee": fee,
                "floor": floor
            }
            results.append(bukken_info)
            item_count += 1

    page_number += 1
    
    next_button = soup.select('p.pagination-parts > a')
    if not next_button or "次へ" not in next_button[-1].text or item_count >= MAX_ITEMS:
        break  # '次へ' ボタンがない、またはアイテム数がMAX_ITEMSに達したらループを終了

# データフレームを作成
df = pd.DataFrame(results)

# CSVに出力
csv_file_path = 'output.csv'
df.to_csv(csv_file_path, index=False)

print(f"CSVファイルが出力されました: {csv_file_path}")


CSVファイルが出力されました: output.csv
